# Examples with Dataframes

> **These are Go notebooks**: In order to use the GoNB Jupyter Kernel, please install GoNB from here: [https://github.com/janpfeifer/gonb](https://github.com/janpfeifer/gonb)

Note also that for local package development, you can put: `!*go mod edit -replace "github.com/umbralcalc/stochadex=/path/to/stochadex"` at the top of any cell.

## To Dataframes

Writing some stored data of a partition to a `gota` dataframe can be done easily using a function in the analysis package. Here's a complete example using data generated from a Poisson process below.

In [ ]:
import (
	"os"

	"github.com/umbralcalc/stochadex/pkg/simulator"
	"github.com/umbralcalc/stochadex/pkg/analysis"
	"github.com/umbralcalc/stochadex/pkg/discrete"
)

%%

// Create a simulator.StateTimeStorage from a simulation run
storage := analysis.NewStateTimeStorageFromPartitions(
	// Instantiate the desired simulation state partitions
	[]*simulator.PartitionConfig{
		{
			Name:              "poisson_data",
			Iteration:         &discrete.PoissonProcessIteration{},
			Params:            simulator.NewParams(map[string][]float64{
				"rates": {0.002, 0.001, 0.004, 0.001},
			}),
			InitStateValues:   []float64{0.0, 0.0, 0.0, 0.0},
			StateHistoryDepth: 1,
			Seed:              123,
		},
    },
	// Decide when should we stop the simulation
	&simulator.NumberOfStepsTerminationCondition{
		MaxNumberOfSteps: 1000,
	},
	// Decide how time should evolve
	&simulator.ConstantTimestepFunction{
		Stepsize: 1000.0,
	},
	// Input the initial time
	1667980544.0,
)

// Retrieve a dataframe representing the data in one partition
df := analysis.GetDataFrameFromPartition(storage, "poisson_data")

// Save the dataframe as a csv for later
file, _ := os.Create("data/test_df.csv")
df.WriteCSV(file)

// Display the dataframe
fmt.Println(df)

## From Dataframes

Getting data into a partition from a `gota` dataframe is also straightforward using another function in the analysis package. Using the Poisson process data, the code below shows how to do this.

In [ ]:
import (
	"os"

    "github.com/go-gota/gota/dataframe"
	"github.com/umbralcalc/stochadex/pkg/simulator"
	"github.com/umbralcalc/stochadex/pkg/analysis"

	"github.com/go-echarts/go-echarts/v2/opts"
	"github.com/go-echarts/go-echarts/v2/charts"
	gonb_echarts "github.com/janpfeifer/gonb-echarts"
)

%%

// Create a dataframe from the file
file, _ := os.Open("data/test_df.csv")
df := dataframe.ReadCSV(file)

// Create new state time storage for the data
storage := simulator.NewStateTimeStorage()

// Add the dataframe data to the storage as a partition
analysis.SetPartitionFromDataFrame(storage, "poisson_data", df, true)

// Reference the plotting data for the x-axis
xRef := analysis.DataRef{Plotting: &analysis.DataPlotting{IsTime: true}}

// Reference the plotting data for the y-axis
yRefs := []analysis.DataRef{{PartitionName: "poisson_data"}}

// Create a scatter plot from partitions in a simulator.StateTimeStorage
scatter := analysis.NewScatterPlotFromPartition(storage, xRef, yRefs)

// Display date-time strings when the time is a UNIX timestamp
scatter.SetGlobalOptions(charts.WithXAxisOpts(opts.XAxis{Type: "time"}))

// Display the plot in a Go notebook
gonb_echarts.Display(scatter, "width: 1024px; height:400px; background: white;")

## Rolling vector aggregations

In order to evaluate the rolling vector aggregations, we need only configure their partitions using functions in the analysis package. Here's another example which differences the Poisson process data and then computes its exponentially-weighted rolling mean and variance below.

In [ ]:
import (
	"github.com/umbralcalc/stochadex/pkg/simulator"
	"github.com/umbralcalc/stochadex/pkg/analysis"
	"github.com/umbralcalc/stochadex/pkg/discrete"

	"gonum.org/v1/gonum/floats"
	"github.com/go-echarts/go-echarts/v2/opts"
	"github.com/go-echarts/go-echarts/v2/charts"
	gonb_echarts "github.com/janpfeifer/gonb-echarts"
)

// Create a function for differencing of the Poisson data to run as a partition 
func diffFunc(
	params *simulator.Params,
	partitionIndex int,
	stateHistories []*simulator.StateHistory,
	timestepsHistory *simulator.CumulativeTimestepsHistory,
) []float64 {
	values := make([]float64, stateHistories[partitionIndex].StateWidth)
	floats.SubTo(values, params.Get("data"), stateHistories[int(
		params.GetIndex("partition", 0))].Values.RawRowView(0))
	return values
}

%%

// Initialise a csv file reader
file, _ := os.Open("data/test_df.csv")

// Create a dataframe from the file
df := dataframe.ReadCSV(file)

// Create new state time storage for the data
storage := simulator.NewStateTimeStorage()

// Add the dataframe data to the storage as a partition
analysis.SetPartitionFromDataFrame(storage, "poisson_data", df, true)

// Configure a partition for differencing of the Poisson data
diffPartition:=	&simulator.PartitionConfig{
	Name:              "diff_poisson_data",
	Iteration:         &general.ValuesFunctionIteration{Function: diffFunc},
	Params:            simulator.NewParams(make(map[string][]float64)),
	ParamsAsPartitions: map[string][]string{
		"partition": {"poisson_data"},
	},
	ParamsFromUpstream: map[string]simulator.NamedUpstreamConfig{
		"data": {Upstream: "poisson_data"},
	},
	InitStateValues:   []float64{0.0, 0.0, 0.0, 0.0},
	StateHistoryDepth: 1,
	Seed:              0,
}

// Run and add the differencing partition to storage
storage = analysis.AddPartitionToStateTimeStorage(
	storage,
	diffPartition,
	map[string]int{"poisson_data": 1},
)

// Configure a partition for computing the exponentially-weighted rolling mean
meanPartition := analysis.NewVectorMeanPartition(
	analysis.AppliedAggregation{
		Name: "diffs_mean",
		Data: analysis.DataRef{PartitionName: "diff_poisson_data"},
		Kernel: &kernels.ExponentialIntegrationKernel{},
	},
	storage,
)

// Set the timescale for the exponential weighting
meanPartition.Params.Set("exponential_weighting_timescale", []float64{70000.0})

// Run and add the mean partition to storage
storage = analysis.AddPartitionToStateTimeStorage(
	storage,
	meanPartition,
	map[string]int{"diff_poisson_data": 100},
)

// Configure a partition for computing the exponentially-weighted rolling variance
variancePartition := analysis.NewVectorVariancePartition(
	analysis.DataRef{PartitionName: "diffs_mean"},
	analysis.AppliedAggregation{
		Name: "diffs_variance",
		Data: analysis.DataRef{PartitionName: "diff_poisson_data"},
		Kernel: &kernels.ExponentialIntegrationKernel{},
	},
	storage,
)

// Set the timescale for the exponential weighting
variancePartition.Params.Set("exponential_weighting_timescale", []float64{70000.0})

// Run and add the variance partition to storage
storage = analysis.AddPartitionToStateTimeStorage(
	storage,
	variancePartition,
	map[string]int{"diffs_mean": 1, "diff_poisson_data": 100},
)

// Reference the plotting data for the x-axis
xRef := analysis.DataRef{
	Plotting: &analysis.DataPlotting{
		IsTime: true,
		TimeRange: &analysis.IndexRange{Lower: 100, Upper: 1000},
	},
}

// Reference the mean partition plotting data for the y-axis
yRefs := []analysis.DataRef{{
	PartitionName: "diffs_mean",
	Plotting: &analysis.DataPlotting{
	    TimeRange: &analysis.IndexRange{Lower: 100, Upper: 1000},
	},
}}

// Create a line plot from partitions in a simulator.StateTimeStorage
line := analysis.NewLinePlotFromPartition(storage, xRef, yRefs, nil)

// Display date-time strings when the time is a UNIX timestamp
line.SetGlobalOptions(charts.WithXAxisOpts(opts.XAxis{Type: "time"}))

// Display the plot in a Go notebook
gonb_echarts.Display(line, "width: 1024px; height:400px; background: white;")

// Reference the variance partition plotting data for the y-axis
yRefs = []analysis.DataRef{{
	PartitionName: "diffs_variance",
	Plotting: &analysis.DataPlotting{
	    TimeRange: &analysis.IndexRange{Lower: 100, Upper: 1000},
	},
}}

// Create another line plot from partitions in a simulator.StateTimeStorage
line = analysis.NewLinePlotFromPartition(storage, xRef, yRefs, nil)

// Display date-time strings when the time is a UNIX timestamp
line.SetGlobalOptions(charts.WithXAxisOpts(opts.XAxis{Type: "time"}))

// Display the plot in a Go notebook
gonb_echarts.Display(line, "width: 1024px; height:400px; background: white;")